# Modelling Data
---

In this note, we will be using all the constructs we have learned so far to "express" the solution of a problem.

### Algebraic Data Type
---

A form of composite type is called `Algebraic Data Type` (ADT), it's formed by combining other types. There are two ways we can combine types, `product` and `sum`. We already have seen the constructs, so let's formalize the concept.

#### product type
---

In [1]:
final case class Perosn(name: String, age: Int)

defined class Perosn

Here, the `Perosn` is an example of `product` type. It is a product of `String` and `Int` type. Another way to put it is, a `Person` type <u>has</u> a `String` <u>and</u> a `Int`. In order to create a `Person` object, we must have a `String` object and an `Int` object.

#### sum type
---

In [2]:
sealed trait Human
final case class Male() extends Human
final case class Female() extends Human

defined trait Human
defined class Male
defined class Female

Here, the `Human` is an example of `sum` type. A `Human` can be either `Male` type or `Female` type. In other words, a `Human` <u>is</u> a `Male` <u>or</u> a `Female`.

The ADT is combination of these two types, product "has-a and" type and sum "is-a or" type.

#### missing pattern
---

What about "is-a and" and "has-a or" pattern? It is possible to express data types with such patterns. 

##### "is-a and"
A class can extend only one class, but many traits. And we use `with` keyword to do that. It called mixin.
So, to express that `A` is a `B` and `C`. We use mixin.

In [3]:
trait B
trait C
trait A extends B with C

defined trait B
defined trait C
defined trait A

- `extends` TYPE, this `TYPE` could be a class or trait.
- `with` TYPE, this could be a trait only, and as much as we like. (... extends B with C with D with E)
    
We use this pattern to share implementation across several classes where it doesn't make sense to make default in the main trait (or class).

##### "has-a or"

To express, `A` has a `B` or `C`. We can use following form.

In [4]:
sealed trait A
final case class D(b: B) extends A
final case class E(c: C) extends A

defined trait A
defined class D
defined class E

We mostly use product type and sum type to express any kind of data in Scala, understanding these patterns is very important for writing idiomatic Scala code.

#### example
---

Let's define a data structure to express a result or a failure of a computation. A result can hold integer value, and failed computation can hold error code and message.

In [5]:
sealed trait Computation
final case class Result(value: Int) extends Computation
final case class Failure(code: Int, msg: String) extends Computation

defined trait Computation
defined class Result
defined class Failure

The ADT `Computation` is readily understandable. The `Computation` is sum type becuase it is either `Result` or `Failure`. The `Result` and `Failure` are product type. `Result` has a `Int` value, and `Failure` has `Int` code and `String` error message.

### Working with data
---

So far we talked about combining existing types to create new types, composite types. In order to write business logic, we need some mechanism which decompose these types, and let us use its component parts. There are two such mechanisms, 

1. subtype polymorphism
2. pattern matching.

#### subtype polymorphism
---

It uses parent-child relation between the types, and allows to pass subtype objects where parenttype object is expected and the behaviour is determined based on the provided concrete type.

In [6]:
sealed trait A {
    def foo: String
}

final case class B() extends A {
    def foo: String = "It's B!"
}

final case class C() extends A {
    def foo: String = "It's C!"
}

defined trait A
defined class B
defined class C

We declared ADT and used subtype polymorphism to implement a method.

In [7]:
val b: A = B()
b.foo

val c: A = C()
c.foo

b: A = B()
res6_1: String = "It's B!"
c: A = C()
res6_3: String = "It's C!"

As you can see, even though the type of `b` and `c` value is `A`, `.foo` returns string based on the concrete instances passed to it. The meaning of polymorphism is, many (poly) forms (morhpism). Here object of type `A`, can behave in many "forms" based on the concrete instance we pass.

We can add an implementation of method in parent trait, and override in subtypes as per requirement. We covered it in `04-trait` note.

Basically, the underlying idea is, by defining a method inside a type. We can use all its component parts along with method arguments to get closer to our desired output.
For example, we can write a method `f` in class `A` which can use `b: B` and `c: C`, the class members and `d: D` method parameter to get the result of type `F`.

```
final case class A(b: B, c: C) {
  override def f(d: D): F = ???
}
```

### pattern matching
---

As we have discussed in the note `03-case-class-pattern-matching`, pattern matching allows use to decompose an object and give access to its sub-parts. We have two chocies here, either (1) define a method in parent trait, which does the pattern matching on its subtypes, or (2) create separate singleton object to place such method in it. 

- the advantage of the later is, we can have muplitple implementation of the method, one per object.
- if method is not using any other value other than component parts, than we can prefer former approach.

### complete example 
---

Let's write ADT for geometric shapes and implement area method using both mechanisms.

```scala
sealed trait Shape
final case class Square(side: Int) extends Shape
final case class Rectangle(length: Int, width: Int) extends Shape
final case class Circle(radius: Int) extends Shape
```

In [8]:
// using polymorphism
import scala.math._

sealed trait Shape {
    def area: Double
}
final case class Square(side: Int) extends Shape {
    def area: Double = pow(side, 2)
}
final case class Rectangle(length: Int, width: Int) extends Shape {
    def area: Double = length * width
}
final case class Circle(radius: Int) extends Shape {
    def area: Double = Pi * pow(radius, 2)
}

import scala.math._


defined trait Shape
defined class Square
defined class Rectangle
defined class Circle

In [9]:
// using patterm matching
import scala.math._

sealed trait Shape {
    /**
     * (1) define a method in parent trait
     */
    def area: Double = this match {
        case Square(side) => pow(side, 2)
        case Rectangle(length, width) => length * width
        case Circle(radius) => Pi * pow(radius, 2)
    }
}
final case class Square(side: Int) extends Shape
final case class Rectangle(length: Int, width: Int) extends Shape
final case class Circle(radius: Int) extends Shape

/**
 * (2) create separate singleton object to place such method in it
 */

object ShapeOps {
    def area(shape: Shape): Double = shape match {
        case Square(side) => pow(side, 2)
        case Rectangle(length, width) => length * width
        case Circle(radius) => Pi * pow(radius, 2)
    }
}

import scala.math._


defined trait Shape
defined class Square
defined class Rectangle
defined class Circle
defined object ShapeOps

### Expression problem
---

Quoted from [wiki](https://en.wikipedia.org/wiki/Expression_problem),

> The expression problem is a new name for an old problem. The goal is to define a datatype by cases, where one can add new cases to the datatype and new functions over the datatype, without recompiling existing code, and while retaining static type safety (e.g., no casts).

In layman's term, a programming language should support some form of mechanism in which extending existing code don't force compiler to re-compile existing code.

Two style of "expression",
1. functional, dumb data and pattern matching
2. object-oriented, object and polymorphism

So, in object-oriented, classes are open to extend, and while adding a new class to a parent doesn't need to make changes in existing code. But if we add methods, we may need to change existing code. In contrast, in functional style adding a new method doesn't need to change existing code. But adding new data will force us to change the existing code base. The example of it would be, imagine a sealed trait and adding a new case class to it, it will break all pattern matching on the traits.

Trade-offs:
1. functional
    - add new method: existing code unchanged
    - add new data: change existing code
2. object-oriented:
    - add new method: change existing code
    - add new data: existing code unchanged
    
We have flexibility to use both polymorphism and pattern matching, and we shold use whichever is appropriate.

### Exercise
---

1) What is `Abstract Data Type`? 

2) Implement ADT for traffic light: red, green and yellow

3) Add `next` method in traffic light ADT which returns subsequent light from "current" light, and implement it using polymorphism.

4) Now implement `next` method using patter matching in parent trait.

5) Explain "expression problem" in your own words. Also explain the trade-offs?